In [76]:
import polars as pl
import polars.selectors as cs
import re

# Using `polars` column selectors

In this notebook, we will look at using [`polars` column selectors](https://docs.pola.rs/api/python/stable/reference/selectors.html#selectors) to perform

1. Column selections,
2. Group & Aggregate, and
3. Table Reshaping

## Topic 1 - The Data - World Bank Economic Indicators

First, let's load the World Bank's [World Development Indicators](https://databank.worldbank.org/source/world-development-indicators).

#### Attempt 1

In [77]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv',
                 ignore_errors=True  # skip over problematic rows

             )
    
)

Country Name,Region,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],1969 [YR1969],1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""",7.6,"""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""",7.6,"""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""",7.6,"""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""",7.6,"""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""

#### Attempt 2

Let's use the first hint and extend the infer schema length.

In [78]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000,
            )
)

Country Name,Region,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],1969 [YR1969],1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""7.6""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""7.6""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""7.6""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""7.6""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""10.9""",""".."""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""","""..""",""".."

#### Attempt 3

Looks like missing data is expressed as `".."`, let's add that as the `null_value`.

In [79]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000, 
             null_values='..',
            )
)

Country Name,Region,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],1969 [YR1969],1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
nu

#### Attempt 4 - Removing rows with a null `Series Name`

Notice that there are some extra rows at the bottom of the table that don't correspond to a series name/code.  Let's remove these.

In [80]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000, 
             null_values='..',
            )
 .filter(pl.col("Series Name").is_not_null())
)

Country Name,Region,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],1969 [YR1969],1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""World""",null,"""Rural population""","""SP.RUR.TOTL""",2.0122e9,2.0244e9,2.0468e9,2.0783e9,2.1094e9,2.1460e9,2.1846e9,2.2221e9,2.2607e9,2.3011e9,2.3423e9,2.3840e9,2.4237e9,2.4628e9,2.4999e9,2.5368e9,2.5716e9,2.6055e9,2.6368e9,2.6659e9,2.6942e9,2.7227e9,2.7545e9,2.7876e9,2.8194e9,2.8513e9,2.8839e9,2.9170e9,2.9500e9,2.9829e9,3.0160e9,3.0469e9,3.0766e9,3.1053e9,3.1326e9,3.1590e9,3.1851e9,3.2100e9,3.2336e9,3.2559e9,3.2766e9,3.2926e9,3.3050e9,3.3163e9,3.3263e9,3.3352e9,3.3440e9,3.3525e9,3.3599e9,3.3672e9,3.3743e9,3.3835e9,3.3941e9,3.4037e9,3.4118e9,3.4182e9,3.4239e9,3.4290e9,3.4325e9,3.4346e9,3.4354e9,3.4324e9,3.4263e9
"""World""",null,"""Urban population""","""SP.U

#### Attempt 5 - Cleaning up the columns names using `rename` and a `dict` comprehension

Notice that there are some extra rows at the bottom of the table that don't correspond to a series name/code.  Let's remove these.

#### Review - the `rename` method

In [81]:
(WB_dev_ind
 .rename({'1960 [YR1960]':'1960',
          '1961 [YR1961]':'1961',
         })
 ).head()

Country Name,Region,Series Name,Series Code,1960,1961,1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],1966 [YR1966],1967 [YR1967],1968 [YR1968],1969 [YR1969],1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null


#### First, do it to one ...

In [82]:
'1960 [YR1960]'.split(' [')

['1960', 'YR1960]']

In [83]:
'1960 [YR1960]'.split(' [')[0]

'1960'

#### ... then do it to all.

In [84]:
(rename_years :=
    {old: old.split(" [")[0] 
     for old in WB_dev_ind.columns 
     if '[YR' in old
    }
)

{'1960 [YR1960]': '1960',
 '1961 [YR1961]': '1961',
 '1962 [YR1962]': '1962',
 '1963 [YR1963]': '1963',
 '1964 [YR1964]': '1964',
 '1965 [YR1965]': '1965',
 '1966 [YR1966]': '1966',
 '1967 [YR1967]': '1967',
 '1968 [YR1968]': '1968',
 '1969 [YR1969]': '1969',
 '1970 [YR1970]': '1970',
 '1971 [YR1971]': '1971',
 '1972 [YR1972]': '1972',
 '1973 [YR1973]': '1973',
 '1974 [YR1974]': '1974',
 '1975 [YR1975]': '1975',
 '1976 [YR1976]': '1976',
 '1977 [YR1977]': '1977',
 '1978 [YR1978]': '1978',
 '1979 [YR1979]': '1979',
 '1980 [YR1980]': '1980',
 '1981 [YR1981]': '1981',
 '1982 [YR1982]': '1982',
 '1983 [YR1983]': '1983',
 '1984 [YR1984]': '1984',
 '1985 [YR1985]': '1985',
 '1986 [YR1986]': '1986',
 '1987 [YR1987]': '1987',
 '1988 [YR1988]': '1988',
 '1989 [YR1989]': '1989',
 '1990 [YR1990]': '1990',
 '1991 [YR1991]': '1991',
 '1992 [YR1992]': '1992',
 '1993 [YR1993]': '1993',
 '1994 [YR1994]': '1994',
 '1995 [YR1995]': '1995',
 '1996 [YR1996]': '1996',
 '1997 [YR1997]': '1997',
 '1998 [YR19

In [85]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000, 
             null_values='..',
            )
 .filter(pl.col("Series Name").is_not_null())
 .rename(rename_years)
).head()

Country Name,Region,Series Name,Series Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6,null,null,null,null,null,null,null,null,null,10.9,null


## Topic 2 - Using list comprehensions to select columns

While there is a cleaner solution (see below), we can use columns selection to illustrate using `list` comprehensions to programmically process the column names in a select.

To illustrate, we look at some familiar string verbs.
- `by_name` to select by the exact name,
- `contains` to check for a sub-string,
- `starts_with` and `ends_with` to select by prefix/suffix,
- `matches` to capture more complicated patterns with a RegEx.

#### Example - Selecting BY NAME

In [86]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if col in ("Series Name", "Series Code")
         ])
).head()

Series Name,Series Code
str,str
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""


#### Example - Selecting the Series name that CONTAINS a substring

In [87]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if 'Series' in col
         ])
).head()


Series Name,Series Code
str,str
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""


#### Example - Selecting that STARTSWITH a substring

In [88]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if col.startswith('199')
         ])
).head()


1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null


#### Example - Selecting columns that MATCH a RegEx pattern

In [89]:
early_90s = re.compile(r'^199[0-4]')
the_90s = [str(y) for y in range(1990, 2000)]
[early_90s.match(y) for y in the_90s]

[<re.Match object; span=(0, 4), match='1990'>,
 <re.Match object; span=(0, 4), match='1991'>,
 <re.Match object; span=(0, 4), match='1992'>,
 <re.Match object; span=(0, 4), match='1993'>,
 <re.Match object; span=(0, 4), match='1994'>,
 None,
 None,
 None,
 None,
 None]

In [90]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if early_90s.match(col)
         ])
).head()


1990,1991,1992,1993,1994
f64,f64,f64,f64,f64
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null


## <font color="red"> Exercise 3.1.1 </font>

Create a table the contains the diabetes prevalence for each region.  For each part, use list comprehensions where possible.

Note that you will need to

1. Reshape to move all relevant series into columns and years into rows,
2. Compute the total diabetes cases for each country/year,
3. Aggregate the numerator (total diabetes) and denominator (population) for each region/year, and
4. Compute the diabetes prevalence.

In [91]:
# Your code here
year_cols = [c for c in WB_dev_ind.columns if c.isdigit()]  # list comprehension as you asked to do in the video

# reshape from wide to long
WB_long = WB_dev_ind.melt(
    id_vars=["Country Name", "Region", "Series Name", "Series Code"], 
    value_vars=year_cols,
    variable_name="Year",
    value_name="Value"
)


C:\Users\im4284hw\AppData\Local\Temp\ipykernel_18972\28510562.py:5: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  WB_long = WB_dev_ind.melt(


In [92]:
# diabetes cases by country and year
df_cases = (
    WB_long
    .filter(pl.col("Series Name") == "Diabetes cases")
    .group_by(["Country Name", "Year"])
    .agg(pl.col("Value").sum().alias("Total_diabetes"))
)


In [93]:
# population by country and year
df_pop = (
    WB_long
    .filter(pl.col("Series Name") == "Population")
    .group_by(["Country Name", "Year"])
    .agg(pl.col("Value").sum().alias("Population"))
)


In [94]:
# join diabetes cases with population
df_country = df_cases.join(df_pop, on=["Country Name", "Year"])
# aggregate to region level
df_region = (
    df_country
    .join(WB_dev_ind.select(["Country Name", "Region"]).unique(), on="Country Name")
    .group_by(["Region", "Year"])
    .agg([
        pl.col("Total_diabetes").sum().alias("Region_diabetes"),
        pl.col("Population").sum().alias("Region_population")
    ])
)


In [95]:
# calculate prevalence
df_result = df_region.with_columns(
    (pl.col("Region_diabetes") / pl.col("Region_population")).alias("Prevalence")
)

df_result.head()


Region,Year,Region_diabetes,Region_population,Prevalence
str,str,f64,f64,f64


## Topic 3 - `polars` Column Selectors

Allow column selection based on

- with familiar string verbs,
- by name,
- index,
- type, or
- other useful helper functions like `contains`, `starts_with`, or `matches`

### Familiar string-based helpers

First, we return to these string verbs.
- `by_name` to select by the exact name,
- `contains` to check for a sub-string,
- `starts_with` and `ends_with` to select by prefix/suffix,
- `matches` to capture more complicated patterns with a RegEx.

#### Example - Selecting by name

In [96]:
(WB_dev_ind
 .select(cs.by_name("Series Name", "Series Code"))
).head()

Series Name,Series Code
str,str
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""


#### Example - Selecting the Series name and code using `contains`

In [97]:
(WB_dev_ind
 .select(cs.contains('Series'))
).head()


Series Name,Series Code
str,str
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""


#### Example - Selecting the 1990's using `starts_with`

In [98]:
(WB_dev_ind
 .select(cs.starts_with('199'))
).head()


1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null


#### Example - Selecting the first five years of 1990's using `matches`

In [99]:
(WB_dev_ind
 .select(cs.matches(r'^199[0-4]'))
).head()


1990,1991,1992,1993,1994
f64,f64,f64,f64,f64
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null
null,null,null,null,null


### Selecting columns by type

One unique feature of `polars` columns selectors is the ability to select columns by type.

#### Example - Selecting by index

In [100]:
(WB_dev_ind
 .select(cs.by_index(range(1,5)))
).head()

Region,Series Name,Series Code,1960
str,str,str,f64
"""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null
"""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null
"""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null
"""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null
"""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS""",null


#### Example - Selecting all string columns.

In [101]:
(WB_dev_ind
 .select(cs.by_dtype(pl.String))
).head()

Country Name,Region,Series Name,Series Code
str,str,str,str
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""


In [102]:
(WB_dev_ind
 .select(cs.string())
).head()

Country Name,Region,Series Name,Series Code
str,str,str,str
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…","""SH.STA.DIAB.ZS"""


### Combining selectors with set operations

Another useful feature: Combine with set operations:

- **Complement.** `~selector1`
- **Union.** `selector1 | selector2`
- **Intersection.** `selector1 & selector2`
- **Difference.** `selector1 - selector2`
- **Symmetric difference.** `selector1 ^ selector2`

#### Example - All the string/index columns excluding the `Series Code`

In [103]:
(WB_dev_ind
 .select(cs.string() - cs.contains('Code'))
).head()


Country Name,Region,Series Name
str,str,str
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…"
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…"
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…"
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…"
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…"


#### Example - All index columns (minus the `Series Code`) and the `1990`s

In [104]:
(WB_dev_ind
 .select(cs.string() - cs.contains('Code') | cs.starts_with('199'))
).head()


Country Name,Region,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…",null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…",null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…",null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…",null,null,null,null,null,null,null,null,null,null
"""Afghanistan""","""Asia""","""Diabetes prevalence (% of popu…",null,null,null,null,null,null,null,null,null,null


### Using column selectors to reshape tables.

Another place where column selectors are useful is when reshaping tables, e.g., using `pivot` or `unpivot`.

### Example - Making a tidy subset of the data

**Goal.** Compare the urban and overall population changes across regions for each year in the `1990`s for each region.

**Task 1.** Tidy up the table by reshaping, by
1. `filter` to the measures of interest,
1. `unpivot` the years in question, and
2. `pivot` to measures into separate columns.

#### Solution - Using columns selectors in `unpivot` and `pivot`

In [105]:
WB_dev_ind['Series Name'].unique()

Series Name
str
"""Urban population"""
"""GDP (constant 2015 US$)"""
"""Diabetes prevalence (% of popu…"
"""Population, total"""
"""Rural population"""
"""Individuals using the Internet…"
"""CO2 emissions (kt)"""
"""Adolescent fertility rate (bir…"


In [106]:
(pop_nums :=
 WB_dev_ind
 .select(cs.string() - cs.contains('Code') 
         | cs.starts_with('199')
        )
 .filter(pl.col('Series Name')
           .str.contains(r'^(Urban|Population)')
        )
 .unpivot(on = cs.starts_with('199'),
          index= cs.string(),
          variable_name = "Year",
          )
 .pivot(on = 'Series Name',
        values = 'value',
        index = cs.by_name('Region', 'Year'),  # Column selectors captured the new column!
        aggregate_function='sum'
       )
)

Region,Year,"Population, total",Urban population
str,str,f64,f64
"""Asia""","""1990""",1.7345e10,5.2656e9
"""Europe""","""1990""",4.2569e9,2.9720e9
"""Africa""","""1990""",3.8158e9,1.2023e9
"""Oceania""","""1990""",1.59420834e8,1.1343462e8
"""The Americas""","""1990""",4.3141e9,3.1286e9
…,…,…,…
"""Africa""","""1999""",4.7785e9,1.6553e9
"""Oceania""","""1999""",1.83646284e8,1.26016476e8
"""The Americas""","""1999""",4.9395e9,3.7764e9


## <font color="red"> Exercise 3.1.2 </font>

Create a table the contains the diabetes prevalence for each region.  For each part, use column selectors where possible.

Note that you will need to

1. Reshape to move all relevant series into columns and years into rows,
2. Compute the total diabetes cases for each country/year,
3. Aggregate the numerator (total diabetes) and denominator (population) for each region/year, and
4. Compute the diabetes prevalence.

In [107]:
# Your code here
# Step 1: Reshape from wide to long
year_cols = WB_dev_ind.select(pl.col("^\\d+$")).columns  # regex match for numeric columns

WB_long = WB_dev_ind.melt(
    id_vars=["Country Name", "Region", "Series Name", "Series Code"],
    value_vars=year_cols,          # now this is a list of strings
    variable_name="Year",
    value_name="Value"
)


C:\Users\im4284hw\AppData\Local\Temp\ipykernel_18972\3206153316.py:5: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  WB_long = WB_dev_ind.melt(


In [108]:
# Step 2: Diabetes cases
df_cases = (
    WB_long
    .filter(pl.col("Series Name") == "Diabetes cases")
    .group_by([pl.col("Country Name"), pl.col("Year")])  # using selectors
    .agg(pl.col("Value").sum().alias("Total_diabetes"))
)

In [109]:
df_pop = (
    WB_long
    .filter(pl.col("Series Name") == "Population")
    .group_by([pl.col("Country Name"), pl.col("Year")])
    .agg(pl.col("Value").sum().alias("Population"))
)


In [110]:
# aggregate by region/year
df_country = df_cases.join(df_pop, on=["Country Name", "Year"])

df_region = (
    df_country
    .join(WB_dev_ind.select([pl.col("Country Name"), pl.col("Region")]).unique(),
          on="Country Name")
    .group_by([pl.col("Region"), pl.col("Year")])
    .agg([
        pl.col("Total_diabetes").sum().alias("Region_diabetes"),
        pl.col("Population").sum().alias("Region_population")
    ])
)

In [111]:
#Prevalence
df_result = df_region.with_columns(
    (pl.col("Region_diabetes") / pl.col("Region_population")).alias("Prevalence")
)

df_result.head()

Region,Year,Region_diabetes,Region_population,Prevalence
str,str,f64,f64,f64


## Topic 4 - Using columns selectors to group and aggregate.

We can also use column selectors when transforming columns, such as

Aggregation also benefits from columns selectors, both for
1. Performing the same calulation on multiple columns,
2. Selecting columns to `group_by`, as well as
2. Performing multiple similar `agg`regations.

### Example 1 - Computing rates per 1,000 people.

First, let's illustrate using a column selector in a mutate.

**Goal.** Compute the per 1,000 person rates for CO2 and GDP for the 2000s.

In [112]:
(per_1000_rates :=
 WB_dev_ind
 .filter(pl.col('Series Name').str.contains(r'^(CO2|GDP|^Pop)'),
         pl.col('Region').is_not_null(),
        )
 .unpivot(on = cs.starts_with('200'),
          index= cs.string() - cs.contains('Code'),
          variable_name = "Year",
          )
 .pivot(on = 'Series Name',
        values = 'value',
        index = cs.by_name('Year', 'Region'),
        aggregate_function='sum',
       )
 .with_columns(cs.starts_with('Pop').cast(pl.Int64))
 .with_columns((cs.float()/pl.col('Population, total')*1000).name.suffix(' per 1,000 people'))
)

Year,Region,"Population, total",CO2 emissions (kt),GDP (constant 2015 US$),"CO2 emissions (kt) per 1,000 people","GDP (constant 2015 US$) per 1,000 people"
str,str,i64,f64,f64,f64,f64
"""2000""","""Asia""",20219573310,4.2543e7,6.0354e13,2.104059,2.9849e6
"""2000""","""Europe""",4295625726,3.5008e7,9.1349e13,8.149786,2.1266e7
"""2000""","""Africa""",4897285710,4.7456e6,7.3803e12,0.969022,1.5070e6
"""2000""","""Oceania""",186439830,2.2406e6,6.0884e12,12.018077,3.2656e7
"""2000""","""The Americas""",5005363314,4.5209e7,1.0959e14,9.032179,2.1894e7
…,…,…,…,…,…,…
"""2009""","""Europe""",4341768852,3.3030e7,1.0483e14,7.607582,2.4144e7
"""2009""","""Africa""",6150365700,6.5194e6,1.1581e13,1.060009,1.8830e6
"""2009""","""Oceania""",218028024,2.5918e6,8.0447e12,11.887546,3.6898e7


### Example 2 - Compute the regional population totals.

#### Solution 1 - Without column selectors

In [113]:
(pop_nums
 .filter(pl.col('Region').is_not_null())
 .group_by('Region', 'Year')
 .agg(pl.col('Population, total').sum(),
      pl.col('Urban population').sum(),
     )
)

Region,Year,"Population, total",Urban population
str,str,f64,f64
"""Europe""","""1992""",4.2808e9,2.9995e9
"""Africa""","""1998""",4.6636e9,1.6007e9
"""Middle East""","""1990""",7.99883298e8,4.7889825e8
"""Middle East""","""1995""",9.14412582e8,5.69331372e8
"""Asia""","""1997""",1.9393e10,6.5832e9
…,…,…,…
"""Oceania""","""1996""",1.7556936e8,1.22203848e8
"""Middle East""","""1991""",8.26120518e8,4.9716129e8
"""Europe""","""1998""",4.2994e9,3.0406e9


#### Solution 2 - With column selectors

In [114]:
(pop_nums_by_region_and_year :=
 pop_nums
 .drop(cs.starts_with('C'))
 .filter(pl.col('Region').is_not_null())
 .group_by(cs.string())
 .agg(cs.float().sum(),
      # cs.float().mean(),
     )
)

Region,Year,"Population, total",Urban population
str,str,f64,f64
"""Europe""","""1991""",4.2709e9,2.9873e9
"""Oceania""","""1994""",1.69968666e8,1.1927238e8
"""Middle East""","""1993""",8.73521784e8,5.35684674e8
"""The Americas""","""1991""",4.3858e9,3.2006e9
"""Africa""","""1999""",4.7785e9,1.6553e9
…,…,…,…
"""Africa""","""1995""",4.3312e9,1.4470e9
"""The Americas""","""1993""",4.5290e9,3.3455e9
"""Middle East""","""1992""",8.52674184e8,5.1825078e8


In [115]:
(pop_nums_by_region_and_year :=
 pop_nums
 .drop(cs.starts_with('C'))
 .filter(pl.col('Region').is_not_null())
 .group_by(cs.string())
 .agg(cs.float().sum().name.prefix('total_'),
      cs.float().mean().name.prefix('mean_'),
     )
)

Region,Year,"total_Population, total",total_Urban population,"mean_Population, total",mean_Urban population
str,str,f64,f64,f64,f64
"""Europe""","""1994""",4.2935e9,3.0181e9,4.2935e9,3.0181e9
"""Middle East""","""1995""",9.14412582e8,5.69331372e8,9.14412582e8,5.69331372e8
"""Africa""","""1996""",4.4414e9,1.4964e9,4.4414e9,1.4964e9
"""Asia""","""1998""",1.9675e10,6.7835e9,1.9675e10,6.7835e9
"""Oceania""","""1992""",1.6516122e8,1.16916756e8,1.6516122e8,1.16916756e8
…,…,…,…,…,…
"""Oceania""","""1993""",1.67485644e8,1.18040172e8,1.67485644e8,1.18040172e8
"""The Americas""","""1996""",4.7364e9,3.5606e9,4.7364e9,3.5606e9
"""Europe""","""1990""",4.2569e9,2.9720e9,4.2569e9,2.9720e9


### Cleaning up multiple column transformations

Finally, we can use column selectors to perform the same computation to multiple columns simultaneously.

### Example - Converting the population totals to per 1000 people. 

#### Solution 1 - Without column selectors

In [116]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year.with_columns(
     (pl.col("total_Population, total") / 1000).alias("Population_total_per_1K"),
     (pl.col("total_Urban population") / 1000).alias("Urban_population_per_1K"),
 )
)


Region,Year,"total_Population, total",total_Urban population,"mean_Population, total",mean_Urban population,Population_total_per_1K,Urban_population_per_1K
str,str,f64,f64,f64,f64,f64,f64
"""Europe""","""1994""",4.2935e9,3.0181e9,4.2935e9,3.0181e9,4293543.3,3.0181e6
"""Middle East""","""1995""",9.14412582e8,5.69331372e8,9.14412582e8,5.69331372e8,914412.582,569331.372
"""Africa""","""1996""",4.4414e9,1.4964e9,4.4414e9,1.4964e9,4.4414e6,1.4964e6
"""Asia""","""1998""",1.9675e10,6.7835e9,1.9675e10,6.7835e9,1.9675e7,6.7835e6
"""Oceania""","""1992""",1.6516122e8,1.16916756e8,1.6516122e8,1.16916756e8,165161.22,116916.756
…,…,…,…,…,…,…,…
"""Oceania""","""1993""",1.67485644e8,1.18040172e8,1.67485644e8,1.18040172e8,167485.644,118040.172
"""The Americas""","""1996""",4.7364e9,3.5606e9,4.7364e9,3.5606e9,4.7364e6,3.5606e6
"""Europe""","""1990""",4.2569e9,2.9720e9,4.2569e9,2.9720e9,4.2569e6,2.9720e6


#### Solution 2 - With column selectors

In [117]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns(cs.float()/1000)
)

Region,Year,"total_Population, total",total_Urban population,"mean_Population, total",mean_Urban population
str,str,f64,f64,f64,f64
"""Europe""","""1994""",4293543.3,3.0181e6,4293543.3,3.0181e6
"""Middle East""","""1995""",914412.582,569331.372,914412.582,569331.372
"""Africa""","""1996""",4.4414e6,1.4964e6,4.4414e6,1.4964e6
"""Asia""","""1998""",1.9675e7,6.7835e6,1.9675e7,6.7835e6
"""Oceania""","""1992""",165161.22,116916.756,165161.22,116916.756
…,…,…,…,…,…
"""Oceania""","""1993""",167485.644,118040.172,167485.644,118040.172
"""The Americas""","""1996""",4.7364e6,3.5606e6,4.7364e6,3.5606e6
"""Europe""","""1990""",4.2569e6,2.9720e6,4.2569e6,2.9720e6


### Example - Standardize multiple columns

#### Solution 1 - Without column selectors

In [119]:
(pop_z_scores_by_region_and_year :=
 pop_nums_by_region_and_year.with_columns(
     (
         (pl.col("total_Population, total") - pl.col("total_Population, total").mean())
         / pl.col("total_Population, total").std()
     ).alias("Population_total_zscore"),
     
     (
         (pl.col("total_Urban population") - pl.col("total_Urban population").mean())
         / pl.col("total_Urban population").std()
     ).alias("Urban_population_zscore"),
 )
)


Region,Year,"total_Population, total",total_Urban population,"mean_Population, total",mean_Urban population,Population_total_zscore,Urban_population_zscore
str,str,f64,f64,f64,f64,f64,f64
"""Europe""","""1994""",4.2935e9,3.0181e9,4.2935e9,3.0181e9,-0.192866,0.27678
"""Middle East""","""1995""",9.14412582e8,5.69331372e8,9.14412582e8,5.69331372e8,-0.737237,-0.909926
"""Africa""","""1996""",4.4414e9,1.4964e9,4.4414e9,1.4964e9,-0.169045,-0.460665
"""Asia""","""1998""",1.9675e10,6.7835e9,1.9675e10,6.7835e9,2.285041,2.101558
"""Oceania""","""1992""",1.6516122e8,1.16916756e8,1.6516122e8,1.16916756e8,-0.85794,-1.129174
…,…,…,…,…,…,…,…
"""Oceania""","""1993""",1.67485644e8,1.18040172e8,1.67485644e8,1.18040172e8,-0.857565,-1.128629
"""The Americas""","""1996""",4.7364e9,3.5606e9,4.7364e9,3.5606e9,-0.121517,0.539671
"""Europe""","""1990""",4.2569e9,2.9720e9,4.2569e9,2.9720e9,-0.198762,0.254445


#### Solution 2 - With columns selectors

In [ ]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns((cs.float() - cs.float().mean())/cs.float().std())
)

Region,Year,"Population, total",Urban population
str,str,f64,f64
"""Oceania""","""1999""",-0.854962,-1.124764
"""Oceania""","""1995""",-0.856729,-1.127355
"""Middle East""","""1996""",-0.733803,-0.901463
"""The Americas""","""1997""",-0.110446,0.5748
"""Oceania""","""1993""",-0.857565,-1.128629
…,…,…,…
"""Africa""","""1997""",-0.151256,-0.435825
"""The Americas""","""1999""",-0.088801,0.644295
"""Middle East""","""1999""",-0.722895,-0.874673


### Example - Standardize multiple columns (within `Region`)

#### Solution 1 - Without column selectors

In [ ]:
(pop_z_scores_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns((pl.col('Population, total') - pl.col('Population, total').mean().over('Region'))/pl.col('Population, total').std().over('Region'), 
               (pl.col('Urban population') - pl.col('Urban population').mean().over('Region'))/pl.col('Urban population').std().over('Region'),
              )
)

Region,Year,"Population, total",Urban population
str,str,f64,f64
"""Oceania""","""1999""",1.496933,1.445769
"""Oceania""","""1995""",0.13435,0.15007
"""Middle East""","""1996""",0.47942,0.480335
"""The Americas""","""1997""",0.82893,0.826205
"""Oceania""","""1993""",-0.509868,-0.487318
…,…,…,…
"""Africa""","""1997""",0.820938,0.812929
"""The Americas""","""1999""",1.467548,1.484156
"""Middle East""","""1999""",1.491771,1.500873


#### Solution 2 - With columns selectors

In [ ]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns((cs.float() - cs.float().mean().over('Region'))/cs.float().std().over('Region'))
)

Region,Year,"Population, total",Urban population
str,str,f64,f64
"""Oceania""","""1999""",1.496933,1.445769
"""Oceania""","""1995""",0.13435,0.15007
"""Middle East""","""1996""",0.47942,0.480335
"""The Americas""","""1997""",0.82893,0.826205
"""Oceania""","""1993""",-0.509868,-0.487318
…,…,…,…
"""Africa""","""1997""",0.820938,0.812929
"""The Americas""","""1999""",1.467548,1.484156
"""Middle East""","""1999""",1.491771,1.500873


## <font color="red"> Exercise 3.1.3 </font>

Now redo the work in the first exercise, but also use one column selector to perform the `agg` step.

In [120]:
# Your code here


# Reshape 
year_cols = [c for c in WB_dev_ind.columns if c.isdigit()]  

WB_long = WB_dev_ind.melt(
    id_vars=["Country Name", "Region", "Series Name", "Series Code"], 
    value_vars=year_cols,
    variable_name="Year",
    value_name="Value"
)

# Compute diabetes cases
df_cases = (
    WB_long
    .filter(pl.col("Series Name") == "Diabetes cases")
    .group_by(["Country Name", "Year"])
    .agg(pl.col("Value").sum().alias("Total_diabetes"))
)

# group and sum population
df_pop = (
    WB_long
    .filter(pl.col("Series Name") == "Population")
    .group_by(["Country Name", "Year"])
    .agg(pl.col("Value").sum().alias("Population"))
)

# Join + aggregate by region/year
df_country = df_cases.join(df_pop, on=["Country Name", "Year"])

df_region = (
    df_country
    .join(WB_dev_ind.select(["Country Name", "Region"]).unique(),
          on="Country Name")
    .group_by(["Region", "Year"])
    # 👇 use ONE selector to aggregate both numeric columns
    .agg(pl.col(["Total_diabetes", "Population"]).sum())
)

# Compute prevalence
df_result = df_region.with_columns(
    (pl.col("Total_diabetes") / pl.col("Population")).alias("Prevalence")
)

df_result.head()


C:\Users\im4284hw\AppData\Local\Temp\ipykernel_18972\3555384290.py:7: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  WB_long = WB_dev_ind.melt(


Region,Year,Total_diabetes,Population,Prevalence
str,str,f64,f64,f64
